In [259]:
import numpy as np
import pandas as pd

# GA Class

In [260]:
class MancalaGA:
    
    def __init__(self, self_state, opponent_state, gen, popsize, generation, pc, pm):
        self.num_of_gen = gen
        self.pop_size = popsize
        self.num_of_generation = generation
        self.pc = pc
        self.pm = pm
        self.self_state = self_state # list state ai (lubang) *default [7,7,7,7,7,7,7]
        self.opponent_state = opponent_state # list state player (lubang) *default [7,7,7,7,7,7,7]
        
    
    def individu(self):
        return np.array([np.random.uniform(0,1) for i in range(self.num_of_gen)])
    
    def population(self):
        return np.array([self.individu() for i in range(self.pop_size)])
    
    def debug_lubang_representation(self):
        
        populasi = self.population()
        
        
        for idx, lubang in enumerate(self.self_state):
            idx_lubang = idx
            isi_lubang = lubang

            # get value tiap heuristik
            h1 = self.h1(idx_lubang, isi_lubang)
            h2 = self.h2(idx_lubang, isi_lubang)
            h3 = self.h3(idx_lubang, isi_lubang)
            h4 = self.h4(idx_lubang, isi_lubang)
            h5 = self.h5(idx_lubang, isi_lubang)


            print(f"ini lubang ke {idx_lubang} dengan isi isi_lubang {isi_lubang}")
            print(f"nilai h1 adalah {h1}")
            print(f"nilai h2 adalah {h2}")
            print(f"nilai h3 adalah {h3}")
            print(f"nilai h4 adalah {h4}")
            print(f"nilai h5 adalah {h5}")
                
                

            print("\n")
                
    def debug(self):
        pass
            
            
    def calc_fitnes(self, individu):
        return np.sum(individu[:self.num_of_gen-1]) - individu[-1]
    
    def self_scan(self):
        data = {}
        for idx, lubang in enumerate(self.self_state):
            h1 = self.h1(idx, lubang)
            h2 = self.h2(idx, lubang)
            h3 = self.h3(idx, lubang)
            h4 = self.h4(idx, lubang)
            h5 = self.h5(idx, lubang)
            
            data[idx] = np.array([h1,h2,h3,h4,h5])
        return data
    
    def candidate(self):
        pass
        
    
    def h1(self, idx_lubang, isi_lubang):
        '''
        Timbun sebanyak mungkin rock dalam satu lubang. 
        Karena pada akhir permainan, rock pada lubang akan dipindahkan semuanya pada lumbung. Lubang kanan semakin aman.
        '''
        # [0,1,2,3,4,5,6] [lumbung]
        if len(self.self_state)-(idx_lubang+isi_lubang) == 0:
            return 1.0
        return 0.0
    
    def h2(self, idx_lubang, isi_lubang):
        '''
        Pertahankan agar rock pada sisi kita sebanyak mungkin. (versi umum H1)
        '''
        if isi_lubang == 0:
            return 0.0
        
        total_batu_before = np.sum(self.self_state)
        
        # simulasikan perpindahan batu pada lubang
        cp_state = self.self_state[:]
        
            
        # pecah list menjadi kiri dan kanan
        kiri = cp_state[:idx_lubang]
        tengah = [0]
        kanan = cp_state[idx_lubang+1:]
        
        jml_batu = isi_lubang
        new_kanan = []
        for i in kanan:
            
            if jml_batu == 0:
                break
            else:
                new_kanan.append(i+1)
            
            jml_batu -= 1
        
        new_state = kiri+tengah+kanan

        
        # jumlahkan setiap batu untuk kondisi terbaru
        total_batu_after = np.sum(new_state)
        
        return total_batu_after/total_batu_before
    
    def h3(self, idx_lubang, isi_lubang):
        '''
        Mempertahankan gerakan memindah rock sebanyak mungkin
        '''
        # [0,1,2,3,4,5,6] [lumbung]
        if len(self.self_state) == (idx_lubang+isi_lubang):
            return 1.0
        return 0.0
    
    def h4(self, idx_lubang, isi_lubang):
        '''
        Memaksimalkan jumlah rock pada lumbung. Dengan memilih langkah mencuri
        '''
        if isi_lubang == 0:
            return 0.0
        
        if 0 not in self.self_state:
            # jika ga ada 0 dalam lubang kita, berarti ga ada bisa yang dicuri
            return 0.0
        
        # jumlah perpindahan batu
        jml_pindah = idx_lubang+isi_lubang
        
        if len(self.self_state) < jml_pindah:
            # out of range
            return 0.0
        
        if self.self_state[jml_pindah] == 0:
            # lakukan pencurian
            jml_curi = self.opponent_state[jml_pindah]
            return 1-(1/jml_curi)
        else:
            # ga ada yang bisa dicuri
            return 0.0
        
    
    def h5(self, idx_lubang, isi_lubang):
        '''
        h5 pada excel dihapus diganti dengan h6 diexcel
        yaitu strategi bertahan
        Menjaga score musuh seminimal mungkin. (heuristik dengan mempertimbangkan 2 gerakan musuh kedepan.
        '''
        # [0,1,2,3,4,5,6] [lumbung]
        if isi_lubang == 0:
            return 0.0
        
        jml_batu_ke_musuh = abs(len(self.self_state) - (idx_lubang+isi_lubang))
        if jml_batu_ke_musuh > 0: # berarti ada batu yang masuk ke lubang musuh
            jml_batu_ke_musuh = len(self.self_state) if jml_batu_ke_musuh > len(self.self_state) else jml_batu_ke_musuh
            return jml_batu_ke_musuh/len(self.self_state)
    
    def turnamen(individu1, individu2):
        pass
    
    def selection(self, population, verbose=False):
        '''
        menggunakan metode turnamen dari individu yang berada dalam kandidat
        '''
        scan = self.self_scan()
        
        kandidat = []
        
        for lubang, heuristic in scan.items():
            if verbose:
                print(f"processing lubang \t: {lubang} dengan nilai heuristic {heuristic}")
                print("\n")
                for individu in population:
                    # h*w
                    total = heuristic*individu
                    print(f"heuristic*weight \t: {total}")
                    print(f"nilai fitness \t\t: {self.calc_fitnes(total)}")
                    print("\n")
                print(f"================================================================================")
                print("\n")
                
                # masukan individu terbaik dari setiap lubang kedalam list kandidat 
            else:
                pass
    
    def crossover(self):
        pass
    
    def mutation(self):
        pass

## RUN

In [261]:
'''
setting param
'''


SELF_STATE = [1,9,9,0,9,9,9]
OPPONENT_STATE = [8,8,8,9,0,8,8]
NUMBER_OF_GEN = 5
NUMBER_OF_POPULATION = 10
NUMBER_OF_GENERATION = 10
PC = 0.8
PM = 0.05


ga = MancalaGA(SELF_STATE, OPPONENT_STATE, NUMBER_OF_GEN, NUMBER_OF_POPULATION, NUMBER_OF_GENERATION, PC, PM)

In [262]:
ga.self_scan()

{0: array([0.        , 0.97826087, 0.        , 0.        , 0.85714286]),
 1: array([0.        , 0.80434783, 0.        , 0.        , 0.42857143]),
 2: array([0.        , 0.80434783, 0.        , 0.        , 0.57142857]),
 3: array([0., 0., 0., 0., 0.]),
 4: array([0.        , 0.80434783, 0.        , 0.        , 0.85714286]),
 5: array([0.        , 0.80434783, 0.        , 0.        , 1.        ]),
 6: array([0.        , 0.80434783, 0.        , 0.        , 1.        ])}

In [263]:
pop = ga.population()
pop

array([[0.99549249, 0.91846844, 0.93196403, 0.33683882, 0.31026988],
       [0.41374798, 0.07581856, 0.57610088, 0.27839758, 0.60044419],
       [0.61467577, 0.41502903, 0.50994974, 0.393956  , 0.37184403],
       [0.14499573, 0.63026799, 0.61311154, 0.39980138, 0.54384908],
       [0.20004911, 0.74977411, 0.22137704, 0.16961704, 0.22360653],
       [0.54824126, 0.09764701, 0.63242219, 0.03770897, 0.51313123],
       [0.62196843, 0.59972526, 0.67850472, 0.46690151, 0.7393982 ],
       [0.93123331, 0.37241931, 0.76439106, 0.9073662 , 0.87075379],
       [0.40821123, 0.54492031, 0.77565962, 0.10210178, 0.94858105],
       [0.508427  , 0.42639087, 0.09599604, 0.44985189, 0.62879226]])

In [264]:
ga.selection(pop, verbose=True)

prosesing lubang 	: 0 dengan nilai heuristic [0.         0.97826087 0.         0.         0.85714286]


heuristic*weight 	: [0.         0.89850174 0.         0.         0.26594561]
nilai fitness 		: 0.6325561253837683


heuristic*weight 	: [0.         0.07417033 0.         0.         0.51466645]
nilai fitness 		: -0.4404961239230456


heuristic*weight 	: [0.         0.40600666 0.         0.         0.31872346]
nilai fitness 		: 0.08728319951354496


heuristic*weight 	: [0.         0.61656651 0.         0.         0.46615635]
nilai fitness 		: 0.15041015795543478


heuristic*weight 	: [0.         0.73347468 0.         0.         0.19166274]
nilai fitness 		: 0.5418119403376344


heuristic*weight 	: [0.         0.09552424 0.         0.         0.43982677]
nilai fitness 		: -0.34430252299423947


heuristic*weight 	: [0.         0.58668775 0.         0.         0.63376988]
nilai fitness 		: -0.04708212783197363


heuristic*weight 	: [0.         0.36432324 0.         0.         0.74636039]
